# calculate classification

there are 5 downstream tasks: gender, sentiment, emotion, age, stance.

use judge model as classifier, input 'predicted_sentence' and output classification in these 5 tasks.  


for gpt model, i use default parameter values,  

temperature = 1


In [120]:
import pandas as pd
import re

import os
from together import Together

from openai import OpenAI
import random
from collections import Counter
import time

In [121]:

prompt_augmentation=False
print(prompt_augmentation)

False


## read dataset with predicted emojis



In [123]:
# # model_name='gpt-4o-mini'
# model_name='llama3.1-70b-instruct'
# model_name = 'llama3.1-8b-instruct'
# model_name = 'gpt-4o'
# model_name = 'gemma2-9b-instruct'
# model_name= 'mistral-7b-instruct'
model_name = 'qwen2-72b-instruct'

# model_name = 'sentence'
# model_name = 'random_sentence'

judge_model_name = "gpt-4o-mini"
# judge_model_name = "gpt-4o-turbo"

print(model_name)
print(judge_model_name)

qwen2-72b-instruct
gpt-4o-mini


In [125]:
!ls ./dataset_predicted/{model_name}/prompt_aug_sentence_cls/dataset_predicted_qwen2-72b-instruct_with_pure_prompt_aug_sentence_cls_5039.csv


# !ls ./dataset_finalOutput/mistral-7b-instruct/dataset_finalOutput_mistral-7b-instruct_gpt-4o-mini_prompt_aug_sentence_cls_5039.csv



./dataset_predicted/qwen2-72b-instruct/prompt_aug_sentence_cls/dataset_predicted_qwen2-72b-instruct_with_pure_prompt_aug_sentence_cls_5039.csv


In [126]:
# dataset_path = f'./dataset_predicted/{model_name}/prompt_aug_sentence_cls/dataset_predicted_gpt-4o_prompt_aug_sentence_cls_5039.csv'

dataset_path = f'./dataset_predicted/{model_name}/prompt_aug_sentence_cls/dataset_predicted_qwen2-72b-instruct_with_pure_prompt_aug_sentence_cls_5039.csv'

# dataset_path = f'./dataset_finalOutput/mistral-7b-instruct/dataset_finalOutput_mistral-7b-instruct_gpt-4o-mini_prompt_aug_sentence_cls_5039.csv'

In [127]:
emoji_dataset = pd.read_csv(dataset_path)

In [128]:
emoji_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5039 entries, 0 to 5038
Data columns (total 33 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   Unnamed: 0.1                                     5039 non-null   int64 
 1   Unnamed: 0                                       5039 non-null   int64 
 2   index                                            5039 non-null   int64 
 3   user_id                                          5039 non-null   object
 4   emoji_sentence                                   5039 non-null   object
 5   char_count                                       5039 non-null   int64 
 6   cluster                                          5039 non-null   int64 
 7   sentence                                         5039 non-null   object
 8   gender_labels                                    5039 non-null   object
 9   sentiment_labels                         

In [80]:
!ls ./dataset_with_logits/gpt-4o/prompt_aug_sentence_cls/emotion/dataset_logits_gpt-4o_prompt_aug_sentence_cls_emotion_2000.csv


./dataset_with_logits/gpt-4o/prompt_aug_sentence_cls/emotion/dataset_logits_gpt-4o_prompt_aug_sentence_cls_emotion_2000.csv


In [81]:

dataset_path  = f'./dataset_with_logits/gpt-4o/prompt_aug_sentence_cls/emotion/dataset_logits_gpt-4o_prompt_aug_sentence_cls_emotion_2000.csv'

emoji_dataset02=pd.read_csv(dataset_path)

In [83]:
emoji_dataset02.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5039 entries, 0 to 5038
Data columns (total 31 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   Unnamed: 0                                  5039 non-null   int64 
 1   index                                       5039 non-null   int64 
 2   user_id                                     5039 non-null   object
 3   emoji_sentence                              5039 non-null   object
 4   char_count                                  5039 non-null   int64 
 5   cluster                                     5039 non-null   int64 
 6   sentence                                    5039 non-null   object
 7   gender_labels                               5039 non-null   object
 8   sentiment_labels                            5039 non-null   object
 9   emotion_labels                              5039 non-null   object
 10  age_labels              

In [87]:
emoji_dataset['emotion_logits_prompt_aug_sentence_cls'] = emoji_dataset02['emotion_logits_prompt_aug_sentence_cls']

In [88]:
emoji_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5039 entries, 0 to 5038
Data columns (total 31 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   Unnamed: 0                                  5039 non-null   int64 
 1   index                                       5039 non-null   int64 
 2   user_id                                     5039 non-null   object
 3   emoji_sentence                              5039 non-null   object
 4   char_count                                  5039 non-null   int64 
 5   cluster                                     5039 non-null   int64 
 6   sentence                                    5039 non-null   object
 7   gender_labels                               5039 non-null   object
 8   sentiment_labels                            5039 non-null   object
 9   emotion_labels                              5039 non-null   object
 10  age_labels              

## classification using judge model

In [129]:
# # model_name='gpt-4o-mini'
# model_name='llama3.1-70b-instruct'
# model_name = 'llama3.1-8b-instruct'
# model_name = 'gpt-4o'
# model_name = 'gemma2-9b-instruct'
# model_name= 'mistral-7b-instruct'
model_name = 'qwen2-72b-instruct'

# model_name = 'sentence'
# model_name = 'random_sentence'

judge_model_name = "gpt-4o-mini"
# judge_model_name = "gpt-4o-turbo"

print(model_name)
print(judge_model_name)
dataset_len=len(emoji_dataset)
print(dataset_len)

qwen2-72b-instruct
gpt-4o-mini
5039


In [130]:

prompt_augmentation=True
print(prompt_augmentation)

True


In [131]:

api_key='' # input your openai api key here

client = OpenAI(api_key=api_key)


In [132]:
emoji_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5039 entries, 0 to 5038
Data columns (total 33 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   Unnamed: 0.1                                     5039 non-null   int64 
 1   Unnamed: 0                                       5039 non-null   int64 
 2   index                                            5039 non-null   int64 
 3   user_id                                          5039 non-null   object
 4   emoji_sentence                                   5039 non-null   object
 5   char_count                                       5039 non-null   int64 
 6   cluster                                          5039 non-null   int64 
 7   sentence                                         5039 non-null   object
 8   gender_labels                                    5039 non-null   object
 9   sentiment_labels                         

In [133]:

# import pandas as pd

# # Assuming 'df' is your DataFrame
# nan_rows = emoji_dataset[emoji_dataset['predicted_emojis_prompt_aug_sentence_cls'].isna()]

# # Display the rows where the column has NaN values
# print(nan_rows.index)



In [134]:
prompts = {
    "sentiment": "I will provide you a tweet. Please classify the sentiment of the tweet as positive, negative, or neutral. Please only output the answer without justification.\nHere is the tweet:\n{text}\n",
    "emotion": "I will provide you a tweet. Please classify the emotion of the tweet as sadness, happiness, fear, anger, surprise, or disgust. Please only output the answer without justification.\nHere is the tweet:\n{text}\n",
    "age": "I will provide you a tweet. Please classify the likely age group of the person who wrote the tweet as 'child', 'teen', 'adult', or 'senior'. Please only output the answer without justification.\nHere is the tweet:\n{text}\n",
    "gender": "I will provide you a tweet. Please classify the likely gender of the person who wrote the tweet as 'male' or 'female'. Please only output the answer without justification.\nHere is the tweet:\n{text}\n",
    "stance": "I will provide you a tweet. Please classify the stance of the tweet as 'favor', 'None', or 'against'. Please only output the answer without justification.\nHere is the tweet:\n{text}\n"
}

def judge_classifier(judge_model_name, task, text):
    prompt = prompts[task].format(text=text)
    
    completion = client.chat.completions.create(
        model=judge_model_name,
        messages=[
            {
                "role": "user",
                "content": prompt
            },
        ],
        temperature=1,
        # max_tokens=256,
        # top_p=1,
        # frequency_penalty=0,
        # presence_penalty=0,
        # response_format={
        # "type": "text"
        # }
    )
    prediction = completion.choices[0].message.content.strip().lower()
    return prediction

In [135]:
print(model_name)
print(judge_model_name)
print(dataset_len)

print(prompt_augmentation)

qwen2-72b-instruct
gpt-4o-mini
5039
True


In [ ]:

task_list = ['gender', 'sentiment', 'emotion', 'age', 'stance']
start_time = time.time() 
# task_list = [ 'emotion']


for task in task_list: 
    
    # Define the folder path
    if prompt_augmentation==True:
        column_name = f'{task}_logits_prompt_aug_sentence_cls'
        
        folder_path = f"./dataset_with_logits/{model_name}/prompt_aug_sentence_cls/{task}"
        os.makedirs(folder_path, exist_ok=True)
    else:
        column_name = f'{task}_logits'
        
        folder_path = f"./dataset_with_logits/{model_name}/{task}"
        os.makedirs(folder_path, exist_ok=True)

    print(f'The classification task is: {task}, the column name is {column_name}:')
    
    logout_len = 200
    left_bound = 0
    
    # if task =='emotion': left_bound=1561 # !!!
    # else: left_bound=0
    
    for i in range(left_bound, dataset_len):
        # if pd.isna(emoji_dataset.at[i, column_name]):
        if model_name == 'qwen2-72b-instruct' or model_name == 'mistral-7b-instruct':
            if prompt_augmentation==True:
                predicted_sentence = emoji_dataset['predicted_pure_sentence_prompt_aug_sentence_cls'][i] # predicted_pure_sentence_prompt_aug_sentence_cls
            else:
                predicted_sentence = emoji_dataset['predicted_pure_sentence'][i]
        else: 
            if prompt_augmentation==True:
                predicted_sentence = emoji_dataset['predicted_sentence_prompt_aug_sentence_cls'][i]
            else: 
                predicted_sentence = emoji_dataset['predicted_sentence'][i]
                
        temp_logits = judge_classifier(judge_model_name, task, predicted_sentence)
        print(i, predicted_sentence, '\n', temp_logits)
        
        emoji_dataset.at[i, column_name] = temp_logits
        
        if (i+1) % logout_len == 0: time.sleep(5)
            
        if (i + 1) % logout_len == 0: 
            end_time = time.time()  
            elapsed_time = end_time - start_time  

            if prompt_augmentation==True:
                output_logits_path = f"./dataset_with_logits/{model_name}/prompt_aug_sentence_cls/{task}/dataset_logits_{model_name}_prompt_aug_sentence_cls_{task}_{i+1}.csv"
            else:
                output_logits_path = f"./dataset_with_logits/{model_name}/{task}/dataset_logits_{model_name}_{task}_{i+1}.csv"

            emoji_dataset.to_csv(output_logits_path, index=False)
            print(f"save path: {output_logits_path}")

            start_time = time.time()
    if prompt_augmentation==True:
        output_logits_path = f"./dataset_with_logits/{model_name}/prompt_aug_sentence_cls/{task}/dataset_logits_{model_name}_prompt_aug_sentence_cls_{task}_{i+1}.csv"
    else:
        output_logits_path = f"./dataset_with_logits/{model_name}/{task}/dataset_logits_{model_name}_{task}_{i+1}.csv"

    # output_logits_path = f"./dataset_with_logits/{model_name}/{task}/dataset_logits_{model_name}_{task}_{dataset_len}.csv"

    emoji_dataset.to_csv(output_logits_path, index=False)
    print(f"save path: {output_logits_path}")
    

The classification task is: gender, the column name is gender_logits_prompt_aug_sentence_cls:
0 Once again he proves that he's for the establishment and not his constituents.  #Shameshame 😠😡😤 
 female
1 More tweets from he  👦❗🤔 
 male
2 I hope the Mexican citizens don't think all Americans are like the POTUS 😔😢😭 
 female
3 C'mon congress get with the program and do something about this  He's giving all a bad wrap. 😠😡🤬 
 male
4 such garbage! Jealousy is a nasty thing  😡😔💔 
 female
5 who lives like that? It's not remotely inspiring  😔😕😠 
 female
6 he's such a damn douche  😡😠😤 
 female
7 I love the  but I can't stand when they have #kellyanneconway on. She never makes any sense  😡😠😤 
 female
8 not nice Perez! You just need to exercise more  😠💪🚫 
 female
9 this makes me I don't understand why people don't get it 😔😕😢 
 female
10 This is real dumb.Dumber than penalties dumb. 😠😡🤯 
 male
11 and he says "our civil discourse is becoming less and less civilised" in response to Women's March, not 

In [117]:
emoji_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5039 entries, 0 to 5038
Data columns (total 38 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   Unnamed: 0.1                                     5039 non-null   int64 
 1   Unnamed: 0                                       5039 non-null   int64 
 2   index                                            5039 non-null   int64 
 3   user_id                                          5039 non-null   object
 4   emoji_sentence                                   5039 non-null   object
 5   char_count                                       5039 non-null   int64 
 6   cluster                                          5039 non-null   int64 
 7   sentence                                         5039 non-null   object
 8   gender_labels                                    5039 non-null   object
 9   sentiment_labels                         

In [118]:
print(prompt_augmentation)

True


In [119]:

folder_path = f'./dataset_finalOutput/{model_name}'
os.makedirs(folder_path, exist_ok=True)

if prompt_augmentation==True: 
    dataset_path = f'./dataset_finalOutput/{model_name}/dataset_finalOutput_{model_name}_{judge_model_name}_prompt_aug_sentence_cls_{dataset_len}.csv'
else:
    dataset_path = f'./dataset_finalOutput/{model_name}/dataset_finalOutput_{model_name}_{judge_model_name}_{dataset_len}.csv'
print(dataset_path)

emoji_dataset.to_csv(dataset_path, index=False)

./dataset_finalOutput/mistral-7b-instruct/dataset_finalOutput_mistral-7b-instruct_gpt-4o-mini_prompt_aug_sentence_cls_5039.csv


## use 'sentence' to get classification

to prove this 5 downstream tasks can demenstrate the semantic preserving performance of model.

i use sentence as input, to get 5 classification, and the accuracy of these classification are lower than the classifications computed using 'predicted-sentence'.

----

use randome sentence to get classification


In [25]:
!ls ./dataset_labels_using_sentence/emotion/dataset_labels_using_sentence_emotion_1400.csv

./dataset_labels_using_sentence/emotion/dataset_labels_using_sentence_emotion_1400.csv


In [15]:
dataset_path = './dataset/balanced_benchmark_0902_5039_with_random_sentence.csv'

In [16]:
emoji_dataset = pd.read_csv(dataset_path)

In [17]:
emoji_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5039 entries, 0 to 5038
Data columns (total 19 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Unnamed: 0                       5039 non-null   int64 
 1   index                            5039 non-null   int64 
 2   user_id                          5039 non-null   object
 3   emoji_sentence                   5039 non-null   object
 4   char_count                       5039 non-null   int64 
 5   cluster                          5039 non-null   int64 
 6   sentence                         5039 non-null   object
 7   gender_labels                    5039 non-null   object
 8   sentiment_labels                 5039 non-null   object
 9   emotion_labels                   5039 non-null   object
 10  age_labels                       5039 non-null   object
 11  stance_labels                    5039 non-null   object
 12  is_duplicate                     5

In [7]:
emoji_dataset.drop('Unnamed: 0.1', axis=1, inplace=True)


In [23]:
# model_name = 'sentence'
model_name = 'random_sentence'


judge_model_name = 'gpt-4o-mini'
dataset_len=len(emoji_dataset)

print(model_name)
print(judge_model_name)
print(dataset_len)

random_sentence
gpt-4o-mini
5039


In [24]:

api_key=''  # input your openai key here
client = OpenAI(api_key=api_key)


In [25]:
prompts = {
    "sentiment": "I will provide you a tweet. Please classify the sentiment of the tweet as positive, negative, or neutral. Please only output the answer without justification.\nHere is the tweet:\n{text}\n",
    "emotion": "I will provide you a tweet. Please classify the emotion of the tweet as sadness, happiness, fear, anger, surprise, or disgust. Please only output the answer without justification.\nHere is the tweet:\n{text}\n",
    "age": "I will provide you a tweet. Please classify the likely age group of the person who wrote the tweet as 'child', 'teen', 'adult', or 'senior'. Please only output the answer without justification.\nHere is the tweet:\n{text}\n",
    "gender": "I will provide you a tweet. Please classify the likely gender of the person who wrote the tweet as 'male' or 'female'. Please only output the answer without justification.\nHere is the tweet:\n{text}\n",
    "stance": "I will provide you a tweet. Please classify the stance of the tweet as 'favor', 'None', or 'against'. Please only output the answer without justification.\nHere is the tweet:\n{text}\n"
}

def judge_classifier(judge_model_name, task, text):
    prompt = prompts[task].format(text=text)
    
    completion = client.chat.completions.create(
        model=judge_model_name,
        messages=[
            {
                "role": "user",
                "content": prompt
            },
        ],
    )
    prediction = completion.choices[0].message.content.strip().lower()
    return prediction

In [ ]:

task_list = ['gender', 'sentiment', 'emotion', 'age', 'stance']
start_time = time.time() 
# task_list = ['emotion', 'age', 'stance']


for task in task_list: 
    if model_name == 'sentence':
        column_name = f'{task}_labels_using_sentence'
    if model_name == 'random_sentence':
        column_name = f'{task}_labels_using_random_sentence'
    else: 
        raise f'error not exist this model_name: {model_name}'
    print(f'The classification task is: {task}, the column name is {column_name}:')
    # logits_list.append([])
    
    # Define the folder path
    folder_path = f"./dataset_with_logits/{model_name}/{task}"
    os.makedirs(folder_path, exist_ok=True)

    logout_len = 200
    left_bound = 0
    
    # if task =='emotion': left_bound=1400 # !!!
    # else: left_bound=0
    
    for i in range(left_bound, dataset_len):
        # if pd.isna(emoji_dataset.at[i, column_name]):
        predicted_sentence = emoji_dataset[f'{model_name}'][i]
        temp_logits = judge_classifier(judge_model_name, task, predicted_sentence)
        print(i, predicted_sentence, '\n', temp_logits)
        
        emoji_dataset.at[i, column_name] = temp_logits
        
        if (i+1) % logout_len == 0: time.sleep(5)
            
        if (i + 1) % logout_len == 0: 
            end_time = time.time()  
            elapsed_time = end_time - start_time 
                        
            output_logits_path = f"./dataset_with_logits/{model_name}/{task}/dataset_labels_using_{model_name}_{task}_{i+1}.csv"

            emoji_dataset.to_csv(output_logits_path, index=False)

            start_time = time.time()
    output_logits_path = f"./dataset_with_logits/{model_name}/{task}/dataset_labels_using_{model_name}_{task}_{dataset_len}.csv"

    emoji_dataset.to_csv(output_logits_path, index=False)
    

The classification task is: gender, the column name is gender_labels_using_random_sentence:
0 Once again he proves that he's for the establishment and not his constituents.  #Shameshame 🦄🐇🌹 
 female
1 More tweets from he  🎶♨️🌌 
 male
2 I hope the Mexican citizens don't think all Americans are like the POTUS 🏆🕯️💞 
 female
3 C'mon congress get with the program and do something about this  He's giving all a bad wrap. ✝️🧢🔝 
 male
4 such garbage! Jealousy is a nasty thing  🚴👅🏆 
 female
5 who lives like that? It's not remotely inspiring  🍿🎧🎈 
 female
6 he's such a damn douche  📌🦊❣️ 
 female
7 I love the  but I can't stand when they have #kellyanneconway on. She never makes any sense  🙀🎂🥓 
 female
8 not nice Perez! You just need to exercise more  🌴🦅☎️ 
 female
9 this makes me I don't understand why people don't get it 🍀🍎🐢 
 female
10 This is real dumb.Dumber than penalties dumb. 👽🍟😑 
 female
11 and he says "our civil discourse is becoming less and less civilised" in response to Women's March,